In [1]:
import sys
sys.path.append("../src")

In [8]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy

from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import statsmodels.api as sm

import urllib.parse
import requests
import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Project Bond King\Curvy-CUSIPs\notebooks\../.env
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [10]:
date = datetime(2024, 10, 31)
sdr = curve_data_fetcher.dtcc_sdr_fetcher.fetch_dtcc_sdr_data_timeseries(
    start_date=date , end_date=date, agency="CFTC", asset_class="RATES"
)
sdr_df = sdr[date]

FETCHING DTCC SDR DATASETS...: 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


Vanilla Receiver Swaptions UPI FISNs: 
- "NA/O Call Epn OIS USD" 
- "NA/O Call Epn Fxd Flt USD"
- "NA/O Call Epn USD"

Vanilla Payer Swaptions UPI FISNs:
- "NA/O P Epn OIS USD"
- "NA/O P Epn Fxd Flt USD"
- "NA/O P Epn USD"

In [18]:
swaption_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_swaption_upis.csv")
fixed_float_swaps_dsb_lookup_df = pd.read_csv(r"C:\Users\chris\Project Bond King\Curvy-CUSIPs\src\CurvyCUSIPs\dsb_upis\dsb_fixed_float_swaps_upis.csv")

In [21]:
swaption_dsb_lookup_df

,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,Derived_ClassificationType,...,Derived_UnderlierName,Derived_UnderlyingAssetType,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_UnderlyingInstrumentUPI,Attributes_NotionalCurrency,Attributes_OptionType,Attributes_OptionExerciseStyle,Attributes_ValuationMethodorTrigger,Attributes_DeliveryType
0,1,Rates,Option,Swaption,UPI,QZ47W4DQ4M1P,New,NaN,2024-11-12T14:42:17,HRACVP,...,NA/Swap Flt Flt EUR,Basis Swap (Float - Float),Bermudan-Call,Physical,QZCLT7R0Q6PC,EUR,CALL,BERM,Vanilla,PHYS
1,1,Rates,Option,Swaption,UPI,QZNH3RMCRP7F,New,NaN,2024-11-12T11:57:16,HRHDVC,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Put,Cash,QZXLBWLQR9W5,USD,PUTO,EURO,Vanilla,CASH
2,1,Rates,Option,Swaption,UPI,QZXFW7S2X4TQ,New,NaN,2024-11-12T11:57:15,HRHAVC,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Cash,QZXLBWLQR9W5,USD,CALL,EURO,Vanilla,CASH
3,1,Rates,Option,Swaption,UPI,QZW12QBSG1TH,New,NaN,2024-11-12T07:32:56,HRCCVP,...,NA/Swap Fxd Flt KRW,Fixed - Floating,Bermudan-Call,Physical,QZ26BPG38C1K,KRW,CALL,BERM,Vanilla,PHYS
4,1,Rates,Option,Swaption,UPI,QZGP67NXL73C,New,NaN,2024-11-12T01:00:36,HRCDVP,...,NA/Swap Fxd Flt JPY,Fixed - Floating,European-Put,Physical,QZWVXZSSBBND,JPY,PUTO,EURO,Vanilla,PHYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,1,Rates,Option,Swaption,UPI,QZZ12MJFMQ9H,New,NaN,2023-11-13T10:24:10,HRCCVP,...,NA/Swap Fxd Flt EUR,Fixed - Floating,Bermudan-Call,Physical,QZ84C8JCVJW1,EUR,CALL,BERM,Vanilla,PHYS
5892,1,Rates,Option,Swaption,UPI,QZPXMTCMNQ46,New,NaN,2023-11-13T10:24:09,HRCCVP,...,NA/Swap Fxd Flt SEK,Fixed - Floating,Bermudan-Call,Physical,QZL4TS9NKTPL,SEK,CALL,BERM,Vanilla,PHYS
5893,1,Rates,Option,Swaption,UPI,QZ928KQPCQRD,New,NaN,2023-11-13T10:24:08,HRCEVP,...,NA/Swap Fxd Flt USD,Fixed - Floating,American-Put,Physical,QZFDML7GL67S,USD,PUTO,AMER,Vanilla,PHYS
5894,1,Rates,Option,Swaption,UPI,QZH3311WT14K,New,NaN,2023-11-13T10:24:08,HRCFVP,...,NA/Swap Fxd Flt SEK,Fixed - Floating,Bermudan-Put,Physical,QZL4TS9NKTPL,SEK,PUTO,BERM,Vanilla,PHYS


In [34]:
underlying_lookup_conds = (
    (fixed_float_swaps_dsb_lookup_df["Header_UseCase"] == "Fixed_Float_OIS") &
    (fixed_float_swaps_dsb_lookup_df["Derived_UnderlyingAssetType"] == "Overnight Index Swap (OIS)") &
    (fixed_float_swaps_dsb_lookup_df["Derived_ShortName"] == "NA/Swap OIS USD") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_NotionalCurrency"] == "USD") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_NotionalSchedule"] == "Constant") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_DeliveryType"] == "PHYS") &
    (fixed_float_swaps_dsb_lookup_df["Attributes_ReferenceRateTermValue"] == 1) &
    (fixed_float_swaps_dsb_lookup_df["Attributes_ReferenceRateTermUnit"] == "DAYS") &
    (fixed_float_swaps_dsb_lookup_df["Derived_UnderlierName"] == "USD-SOFR-OIS Compound")   
)

underlying_row = fixed_float_swaps_dsb_lookup_df[underlying_lookup_conds]
assert len(underlying_row) == 1
underlying_dict = underlying_row.to_dict("records")[0]
underlying_dict

{'Unnamed: 0': 15053,
 'TemplateVersion': 1,
 'Header_AssetClass': 'Rates',
 'Header_InstrumentType': 'Swap',
 'Header_UseCase': 'Fixed_Float_OIS',
 'Header_Level': 'UPI',
 'Identifier_UPI': 'QZPB5VSBGRCD',
 'Identifier_Status': 'New',
 'Identifier_StatusReason': nan,
 'Identifier_LastUpdateDateTime': '2023-10-15T13:20:35',
 'Derived_ClassificationType': 'SRHCSP',
 'Derived_UnderlyingAssetType': 'Overnight Index Swap (OIS)',
 'Derived_SingleorMultiCurrency': 'Single Currency',
 'Derived_CFIDeliveryType': 'Physical',
 'Derived_ShortName': 'NA/Swap OIS USD',
 'Derived_UnderlierName': 'USD-SOFR-OIS Compound',
 'Attributes_NotionalCurrency': 'USD',
 'Attributes_ReferenceRate': 'USD-SOFR-OIS Compound',
 'Attributes_ReferenceRateTermValue': 1,
 'Attributes_ReferenceRateTermUnit': 'DAYS',
 'Attributes_NotionalSchedule': 'Constant',
 'Attributes_DeliveryType': 'PHYS'}

In [ ]:
epn_ois_sofr_swaptions_lookup_conds = ()

,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,Derived_ClassificationType,...,Derived_UnderlierName,Derived_UnderlyingAssetType,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_UnderlyingInstrumentUPI,Attributes_NotionalCurrency,Attributes_OptionType,Attributes_OptionExerciseStyle,Attributes_ValuationMethodorTrigger,Attributes_DeliveryType
0,1,Rates,Option,Swaption,UPI,QZPJP912N3MF,New,NaN,2024-11-10T17:46:18,HRCFVC,...,NA/Swap Fxd Flt AED,Fixed - Floating,Bermudan-Put,Cash,QZSB3837MG7W,AED,PUTO,BERM,Vanilla,CASH
1,1,Rates,Option,Swaption,UPI,QZ24Q2G51MDS,New,NaN,2024-11-10T12:41:36,HRCFVC,...,NA/Swap Fxd Flt USD,Fixed - Floating,Bermudan-Put,Cash,QZRB0HMK9NWH,USD,PUTO,BERM,Vanilla,CASH
2,1,Rates,Option,Swaption,UPI,QZ0ZVWS4DQDT,New,NaN,2024-11-10T12:41:16,HRCFVC,...,NA/Swap Fxd Flt USD,Fixed - Floating,Bermudan-Put,Cash,QZ86HGZ48ZCS,USD,PUTO,BERM,Vanilla,CASH
3,1,Rates,Option,Swaption,UPI,QZSZ062BTT6D,New,NaN,2024-11-10T12:38:06,HRCFVC,...,NA/Swap Fxd Flt USD,Fixed - Floating,Bermudan-Put,Cash,QZG86MKJJVC3,USD,PUTO,BERM,Vanilla,CASH
4,1,Rates,Option,Swaption,UPI,QZ84T92LJKB7,New,NaN,2024-11-09T10:50:56,HRMAVP,...,NA/Rts Swaps Oth GBP,Other,European-Call,Physical,QZ41LPV34PNB,GBP,CALL,EURO,Vanilla,PHYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5882,1,Rates,Option,Swaption,UPI,QZQ0XGG4F6WK,New,NaN,2023-11-05T12:32:14,HRHDVP,...,NA/Swap OIS INR,Overnight Index Swap (OIS),European-Put,Physical,QZPB1RXQ05RR,INR,PUTO,EURO,Vanilla,PHYS
5883,1,Rates,Option,Swaption,UPI,QZX47P7LVKD6,New,NaN,2023-11-05T12:32:14,HRCAVP,...,NA/Swap Fxd Flt EUR,Fixed - Floating,European-Call,Physical,QZFXR5M3KKN0,EUR,CALL,EURO,Vanilla,PHYS
5884,1,Rates,Option,Swaption,UPI,QZ0MB79S3635,New,NaN,2023-11-05T12:32:14,HRHGVP,...,NA/Swap OIS JPY,Overnight Index Swap (OIS),European-Chooser,Physical,QZV3WV3G4N80,JPY,OPTL,EURO,Vanilla,PHYS
5885,1,Rates,Option,Swaption,UPI,QZL02RQQ63WQ,New,NaN,2023-11-05T12:04:13,HRCDVP,...,NA/Swap Fxd Flt ZAR,Fixed - Floating,European-Put,Physical,QZ2HC7KH0W9L,ZAR,PUTO,EURO,Vanilla,PHYS


In [ ]:
epn_ois_sofr_receivers_conds = (
    (sdr_df["UPI FISN"] == "NA/O Call Epn OIS USD") &
    (sdr_df["Non-standardized term indicator"] == False) &
)


swaption_columns = [
    "Dissemination Identifier",
    "Action type",
    "Event type",
    "Event timestamp",
    "Execution Timestamp",
    "Effective Date",
    "Expiration Date",
    "Maturity date of the underlier",
    "Non-standardized term indicator",
    "Platform identifier",
    "Notional amount-Leg 1",
    "Notional amount-Leg 2",
    "Notional amount in effect on associated effective date-Leg 2",
    "Effective date of the notional amount-Leg 2",
    "First exercise date",
    "Fixed rate-Leg 1",
    "Fixed rate-Leg 2",
    "Option Premium Amount",
    "Strike Price",
    "Fixed rate day count convention-leg 1",
    "Fixed rate day count convention-leg 2",
    "Floating rate day count convention-leg 1",
    "Floating rate day count convention-leg 2",
    "Floating rate reset frequency period-leg 1",
    "Floating rate reset frequency period-leg 2",
    "Floating rate reset frequency period multiplier-leg 1",
    "Floating rate reset frequency period multiplier-leg 2",
    "Other payment amount",
    "Fixed rate payment frequency period-Leg 1",
    "Floating rate payment frequency period-Leg 1",
    "Fixed rate payment frequency period-Leg 2",
    "Floating rate payment frequency period-Leg 2",
    "Fixed rate payment frequency period multiplier-Leg 1",
    "Floating rate payment frequency period multiplier-Leg 1",
    "Fixed rate payment frequency period multiplier-Leg 2",
    "Floating rate payment frequency period multiplier-Leg 2",
    "Other payment type",
    "Embedded Option type",
    "Option Type",
    "Option Style",
    "Package indicator",
    "Unique Product Identifier",
    "UPI FISN",
    "UPI Underlier Name",
]

In [20]:
epn_ois_sofr_receivers_df = sdr_df[sdr_df["UPI FISN"] == "NA/O Call Epn OIS USD"][swaption_columns]
epn_ois_sofr_receivers_upis = set(epn_ois_sofr_receivers_df["Unique Product Identifier"].to_numpy())

In [22]:
swaption_dsb_lookup_df[swaption_dsb_lookup_df["Identifier_UPI"].isin(list(epn_ois_sofr_receivers_upis))]

,TemplateVersion,Header_AssetClass,Header_InstrumentType,Header_UseCase,Header_Level,Identifier_UPI,Identifier_Status,Identifier_StatusReason,Identifier_LastUpdateDateTime,Derived_ClassificationType,...,Derived_UnderlierName,Derived_UnderlyingAssetType,Derived_CFIOptionStyleandType,Derived_CFIDeliveryType,Attributes_UnderlyingInstrumentUPI,Attributes_NotionalCurrency,Attributes_OptionType,Attributes_OptionExerciseStyle,Attributes_ValuationMethodorTrigger,Attributes_DeliveryType
3605,1,Rates,Option,Swaption,UPI,QZ5PFDN726FD,New,NaN,2023-11-17T13:30:18,HRHAVP,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZBTG2KLN231,USD,CALL,EURO,Vanilla,PHYS
4368,1,Rates,Option,Swaption,UPI,QZQ0XDZ8GTRP,New,NaN,2023-11-15T18:38:20,HRHAVP,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZBSN88SSD9S,USD,CALL,EURO,Vanilla,PHYS
5443,1,Rates,Option,Swaption,UPI,QZZWMTRWC4PN,New,NaN,2023-11-10T07:05:11,HRHAVC,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Cash,QZ749DHWD023,USD,CALL,EURO,Vanilla,CASH
5597,1,Rates,Option,Swaption,UPI,QZVLJBR2Z1VS,New,NaN,2023-11-06T21:29:19,HRHAVC,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Cash,QZKQ8QSWZKR2,USD,CALL,EURO,Vanilla,CASH
5608,1,Rates,Option,Swaption,UPI,QZ1SC5570DC7,New,NaN,2023-11-06T17:26:21,HRHAVC,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Cash,QZPB5VSBGRCD,USD,CALL,EURO,Vanilla,CASH
5633,1,Rates,Option,Swaption,UPI,QZZLNQ2D4JQT,New,NaN,2023-11-06T15:25:48,HRHAVP,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZKQ8QSWZKR2,USD,CALL,EURO,Vanilla,PHYS
5647,1,Rates,Option,Swaption,UPI,QZZGWPNBF5R3,New,NaN,2023-11-06T13:17:02,HRHAVP,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZPB5VSBGRCD,USD,CALL,EURO,Vanilla,PHYS
5721,1,Rates,Option,Swaption,UPI,QZKWWN6SSFDH,New,NaN,2023-11-06T04:41:57,HRHAVP,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZT3ZL26QNGW,USD,CALL,EURO,Vanilla,PHYS
5869,1,Rates,Option,Swaption,UPI,QZBK1N7NJ0VF,New,NaN,2023-11-05T12:32:17,HRHAVP,...,NA/Swap OIS USD,Overnight Index Swap (OIS),European-Call,Physical,QZ749DHWD023,USD,CALL,EURO,Vanilla,PHYS
